<a href="https://colab.research.google.com/github/RafaelCaballero/Julio24/blob/main/code/23Hiperpar%C3%A1metros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la ciencia de datos con Python
Rafa Caballero

## Ajuste de hiperparámetros

### Índice
[Ajuste de parámetros uno a uno](#amano)<br>
[GridSearchCV](#GridSearchCV)<br>
[RandomizedSearchCV](#RandomizedSearchCV)<br>

Empezamos cargando las librerías y los datos que vamos a usar

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, train_test_split,cross_val_predict
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import cohen_kappa_score,make_scorer,confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

path = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/movimiento.csv"
import pandas as pd

df = pd.read_csv(path)

df

Se trata de datos tomados de los sensores de móviles en periodos de pocos segundos, nuestro objetivo es averiguar la actividad que está realizando la persona que lleva el móvil

In [ ]:
df.info()

**Ejercicio** Ver cuántos valores diferentes toma df.target y la frecuencia de cada uno

No tendremos que usar nungún tipo de sampling (over, under, SMOTE...), está muy equilibrado. Aunque no lo hagamos en detalle habría que ver histogramas, outlayers y correlaciones:

In [ ]:
df.accelerometer_mean.hist(bins=30)
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

correlaciones = df.corr(numeric_only=True)
g = sns.clustermap(correlaciones,
                   method = 'complete',
                   cmap   = 'RdBu',
                   annot  = True,
                   annot_kws = {'size': 8})
plt.show()



<a name="amano"></a>
#### Ajuste de los parámetros uno a uno

Empleamos KNN para adivinar el tipo de movimiento

In [ ]:

etiquetas = ['Quieto','Coche','Tren','Autobús','Andando']

#1
yColumn="label"
XColumns = [c for c in df.columns if c!=yColumn and c!='target']

X = df[XColumns]
y = df[yColumn]

# 2
test = 0.4
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

# 3
steps = [('StandardScaler', StandardScaler()),('KNN', KNeighborsClassifier())]
metodo = Pipeline(steps=steps)
modelo = metodo.fit(X_train,y_train)

# 4
y_pred = modelo.predict(X_test)

kappa= cohen_kappa_score(y_test,y_pred)
print("Kappa ",kappa)

cm = confusion_matrix(y_test, y_pred, labels=modelo.classes_,normalize="true")
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=etiquetas)
disp.plot()
plt.show()
print(classification_report(y_test, y_pred))

Kappa con validación cruzada

In [ ]:
scorer = make_scorer(cohen_kappa_score)

steps = [('StandardScaler', StandardScaler()),('KNN', KNeighborsClassifier())]
pipeline = Pipeline(steps=steps)
cv = RepeatedStratifiedKFold(n_splits=20, n_repeats=10)
scores = cross_val_score(pipeline, X, y, scoring=scorer, cv=cv)
kappa= scores.mean()
print("Kappa ",kappa)


**Ejercicio** En el código anterior probar a cambiar `StandardScaler()` por `MinMaxScaler()`, y también por nada, elegir la mejor opción

Probamos a quitar algunas de las columnas correlacionadas

In [ ]:
borrar = ["accelerometer_max","gyroscope_max","gyroscope_std","sound_max","sound_min"]
X2 = X.drop(columns=borrar)
steps = [('StandardScaler', StandardScaler()),('KNN', KNeighborsClassifier())]
scores = cross_val_score(pipeline, X2, y, scoring=scorer, cv=cv)
kappa= scores.mean()
print("Kappa ",kappa)

Empeora ligeramente, pero simplifica el dataframe y puede dar buen resultado con otros métodos.


In [ ]:
X=X2

Vamos a determinar el mejor valor de $k$

In [ ]:

ks = []
kappas = []
for k in range(1,12):
    steps = [('StandardScaler', StandardScaler()),('KNN', KNeighborsClassifier(n_neighbors=k))]
    pipeline = Pipeline(steps=steps)
    cv = RepeatedStratifiedKFold(n_splits=20, n_repeats=10)
    scores = cross_val_score(pipeline, X, y, scoring=scorer, cv=cv)
    kappa= scores.mean()
    print(k,kappa)
    ks.append(k)
    kappas.append(kappa)

In [ ]:
plt.plot(ks,kappas)
plt.show()

Por tanto elegimos k=3. Ahora tenemos que elegir la distancia. Utilizar k=3, no hace falta gráfica (no tiene sentido)

In [ ]:
distances = ['minkowski','cosine', 'chebyshev', 'euclidean',  'manhattan']


Vamos a ver si merece la pena considerar con más peso a los vecinos más cercanos o solo "contar"

In [ ]:
weights = ['uniform', 'distance']
for w in weights:
    steps = [('StandardScaler', StandardScaler()),('KNN', KNeighborsClassifier(n_neighbors=3,metric='manhattan',weights=w))]
    pipeline = Pipeline(steps=steps)
    cv = RepeatedStratifiedKFold(n_splits=20, n_repeats=10)
    scores = cross_val_score(pipeline, X, y, scoring=scorer, cv=cv)
    kappa= scores.mean()
    print(w,kappa)


Recapitulamos comparando, primero sin hiperparámetros y luego con los que hemos encontrado

In [ ]:
etiquetas = ['Quieto','Coche','Tren','Autobús','Andando']

#1
yColumn="label"
XColumns = [c for c in df.columns if c!=yColumn and c!='target' and c not in borrar]

X = df[XColumns]
y = df[yColumn]

# 2
test = 0.4
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

# 3
steps = [('StandardScaler', StandardScaler()),('KNN', KNeighborsClassifier())]
metodo = Pipeline(steps=steps)
modelo = metodo.fit(X_train,y_train)

# 4
y_pred = modelo.predict(X_test)

kappa= cohen_kappa_score(y_test,y_pred)
print("Kappa ",kappa)

cm = confusion_matrix(y_test, y_pred, labels=modelo.classes_,normalize="true")
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=etiquetas)
disp.plot()
plt.show()
print(classification_report(y_test, y_pred))


In [ ]:
# 3
steps = [('StandardScaler', StandardScaler()),('KNN', KNeighborsClassifier(n_neighbors=3,metric='manhattan',weights='distance'))]
metodo = Pipeline(steps=steps)
modelo = metodo.fit(X_train,y_train)

# 4
y_pred = modelo.predict(X_test)

kappa= cohen_kappa_score(y_test,y_pred)
print("Kappa ",kappa)

cm = confusion_matrix(y_test, y_pred, labels=modelo.classes_,normalize="true")
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=etiquetas)
disp.plot()
plt.show()
print(classification_report(y_test, y_pred))

<a name="GridSearchCV"></a>
#### GridSearchCV

Como vemos el método ha sido muy laborioso. Además nada nos asegura que si hubiéramos puesto "manhattan" al principio, por ejemplo, no hubiera salido un k diferente. Si queremos asegurar de que eso no pasa tenemos que probar ¡todas las combinaciones de todos los parámetros enter sí! Así aseguraremos que elegimos la mejor combinación, pero con un coste en tiempo altísimo.

De eso se encarga GridSearchCV


In [ ]:
!pip install ipython-autotime
%load_ext autotime

In [ ]:
from sklearn.model_selection import GridSearchCV

paramGrid = {'KNN__n_neighbors': list(range(1,12)),
             'KNN__metric': ['minkowski','cosine', 'chebyshev', 'euclidean',  'manhattan'],
             'KNN__weights': ['uniform', 'distance']
            }

steps = [('StandardScaler', StandardScaler()),('KNN', KNeighborsClassifier())]
pipeline = Pipeline(steps=steps)

metodo = GridSearchCV(pipeline, paramGrid,cv=cv,scoring=scorer,n_jobs=-1,refit=True)
modelo = metodo.fit(X,y)




Podemos ver el mejor kappa, con qué parámetros se consigue y cuál es el pipeline asociado

In [ ]:
print(modelo.best_score_)
print(modelo.best_params_)
print(modelo.best_estimator_)

Hemos puesto `refit=True` para que tras encontrar los mejores parámetros haga un último modelo con el conjunto completo y esos valores. Por eso `best_estimator_` es el mejor modelo con esos parámetros. Podemos grabarlo

In [ ]:
from joblib import dump, load
estimator = modelo.best_estimator_
dump(estimator, "modelo_sensores.joblib")


Ahora en otro notebook podríamos hacer por ejemplo

In [ ]:
from joblib import dump, load
model_sensor = load("modelo_sensores.joblib")

#        accelerometer_mean accelerometer_min accelerometer_std  gyroscope_mean  gyroscope_min  sound_mean  sound_std
valor = [10.00,             9.00,              0.50,             0.183202,       0.020667,      89.770732,   0.006389]

etiquetas = ['Quieto','Coche','Tren','Autobús','Andando']
prediccion = model_sensor.predict([valor])
print(etiquetas[prediccion[0]])
print(model_sensor.predict_proba([valor]))

<a name="RandomizedSearchCV"></a>
#### RandomizedSearchCV

La búsqueda "en rejilla" puede ser muy lenta; en el caso anterior hay 12x5x2=60, y comoen cv tenemos 20 particiones y 10 repeticiones, quedan 12000 modelos a entrenar. Para reducir este tiempo podemos utilizar `RandomizedSearchCV`

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

metodo = RandomizedSearchCV(pipeline, paramGrid,cv=cv,scoring=scorer,n_jobs=-1,n_iter=25)
modelo = metodo.fit(X,y)

In [ ]:
print(modelo.best_score_)
print(modelo.best_params_)
print(modelo.best_estimator_)